In [3]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.pinecone import Pinecone
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint as HuggingFaceHub
from dotenv import load_dotenv
from pinecone import Pinecone as pc
from pinecone import PodSpec
import os

from data_chunking import datachunk
from langchain.retrievers.multi_query import MultiQueryRetriever

In [58]:
embeddings = HuggingFaceEmbeddings()
    
pinecone_instance = pc(api_key=os.getenv('PINECONE_API_KEY'), embeddings=embeddings)

index_name = "eerbot"
environment = "gcp-starter"
spec = PodSpec(environment=environment)

docsearch = Pinecone.from_existing_index(index_name, embeddings)

In [3]:
# Meta tag vector store
embeddings = HuggingFaceEmbeddings()
    
pinecone_instance = pc(api_key=os.getenv('PINECONE_API_KEY'), embeddings=embeddings)

index_name = "eerbot-meta"
environment = "gcp-starter"
spec = PodSpec(environment=environment)

In [60]:
from langchain_community.document_loaders import CSVLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datetime import datetime, timedelta
from timescale_vector import client
import os

def datachunk(transcripts_folder,pdf_folder):
    def create_uuid(date_string: str):
        if date_string is None:
            return None
        time_format = '%a %b %d %H:%M:%S %Y'
        date_string = date_string.strip()  # Remove leading and trailing whitespace
        datetime_obj = datetime.strptime(date_string, time_format)
        uuid = client.uuid_from_time(datetime_obj)
        return str(uuid)
    
    csv_files = [file for file in os.listdir(transcripts_folder) if file.endswith('.csv')]
    pdf_files = [file for file in os.listdir(pdf_folder) if file.endswith('.pdf')]

    documents = []
    
    # Load text documents from .pdf files
    for pdf_file in pdf_files:
        loader = PyPDFLoader(os.path.join(pdf_folder, pdf_file))
        documents.extend(loader.load())

    print("PDF files loaded...")

    # Load text documents from .txt files
    for csv_file in csv_files:
        loader = CSVLoader(
            os.path.join(transcripts_folder, csv_file),
            csv_args={"delimiter": ";"},
            metadata_columns=["speaker_name", "date_time"])
        loaded_documents = loader.load()
        
        for doc in loaded_documents:
            date_string = doc.metadata.get("date_time") 
            if date_string:
                uuid = create_uuid(date_string)
                doc.metadata["uuid"] = uuid
                
        documents.extend(loaded_documents)

    print("Text files loaded...")


    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)
    
    return docs

In [79]:
transcripts_folder = os.path.join("..", "data","reformatted_transcripts")
pdf_folder = os.path.join("..", "data","EER-site-pages-pdf")
docs = datachunk(transcripts_folder,pdf_folder)

PDF files loaded...
Text files loaded...


In [71]:
llm = HuggingFaceHub(
            repo_id="mistralai/Mistral-7B-Instruct-v0.2",
            temperature=1.0,
            top_p=0.8,
            top_k=50,
            huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY')
        )

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\szh\.cache\huggingface\token
Login successful


In [62]:
docs

[Document(page_content="'EER and the More-than-human' workshopOrganised by Andreas Roepstorff, Anna Engberg-Pedersen, Joe Dumit, Olafur Eliasson, SophieErlundParticipants Amos Blanton, Amy Cutler, Andreas Løppenthin, Charlotte Tornøe EkkelundNørholm, Christine Fentz, Cordula Vesper, Dorte Bjerre Jensen, Helene Nymann, Julie KampeZiska, Karen Grøn, Karsten Olsen, Katrin Heimann, Narendran K & Krishnapriya CP,", metadata={'source': "..\\data\\EER-site-pages-pdf\\EER 'EER and the More-than-human' workshop.pdf", 'page': 0}),
 Document(page_content='Nymann, Julie KampeZiska, Karen Grøn, Karsten Olsen, Katrin Heimann, Narendran K & Krishnapriya CP, NatashaMyers, Neema Githere, Peter Dalsgaard, Rune Bosse, Elizabeth McTernan31 Aug → 01 Sep 2021joint actionmicro-phenomenologymore-than-humandecision makingdrawingreﬂectionexperiment', metadata={'source': "..\\data\\EER-site-pages-pdf\\EER 'EER and the More-than-human' workshop.pdf", 'page': 0}),
 Document(page_content="Back'EER and the More-than

In [92]:
# create a sample of the docs
docs_sample = docs[0:15]
docs_sample

[Document(page_content="'EER and the More-than-human' workshopOrganised by Andreas Roepstorff, Anna Engberg-Pedersen, Joe Dumit, Olafur Eliasson, SophieErlundParticipants Amos Blanton, Amy Cutler, Andreas Løppenthin, Charlotte Tornøe EkkelundNørholm, Christine Fentz, Cordula Vesper, Dorte Bjerre Jensen, Helene Nymann, Julie KampeZiska, Karen Grøn, Karsten Olsen, Katrin Heimann, Narendran K & Krishnapriya CP,", metadata={'source': "..\\data\\EER-site-pages-pdf\\EER 'EER and the More-than-human' workshop.pdf", 'page': 0}),
 Document(page_content='Nymann, Julie KampeZiska, Karen Grøn, Karsten Olsen, Katrin Heimann, Narendran K & Krishnapriya CP, NatashaMyers, Neema Githere, Peter Dalsgaard, Rune Bosse, Elizabeth McTernan31 Aug → 01 Sep 2021joint actionmicro-phenomenologymore-than-humandecision makingdrawingreﬂectionexperiment', metadata={'source': "..\\data\\EER-site-pages-pdf\\EER 'EER and the More-than-human' workshop.pdf", 'page': 0}),
 Document(page_content="Back'EER and the More-than

In [86]:
# Define the prompt template
prompt_template = PromptTemplate(
     template="Generate 5 keywords that accurately describe the provided text. Only return the keywords in a single line, separated by commas, with no extra text. Here is the text: '{metatext}'",
    input_variables=["metatext"]
)

In [ ]:
How does it relate to how cultural emisions, 

In [90]:
prompt = prompt_template.format(metatext=docs_sample[1].page_content)

In [91]:
llm(prompt)

'\n\ngreeting, help, Bible, turn on piece, aspect'

In [ ]:
# Function to generate text using the LLM
def generate_text_for_docs(docs_sample, prompt_template, llm):
    for doc in docs_sample:
        prompt = prompt_template.format(metatext=doc.page_content)
        generated_text = llm(prompt)
        doc.page_content = generated_text

# Generate text for the sample documents
generate_text_for_docs(docs_sample, prompt_template, llm)

In [95]:
docs_sample[1].page_content = "The quick brown fox jumps over the lazy dog"

In [96]:
docs_sample[1].page_content

'The quick brown fox jumps over the lazy dog'

In [75]:
docs[7000:7005]

[Document(page_content=', larger history of casting.', metadata={'source': '..\\data\\reformatted_transcripts\\2022-09-29 11.01.46_rf.csv', 'row': 49, 'speaker_name': 'Katrin Heimann', 'date_time': 'Thu Sep 29 00:26:28 2022 ', 'uuid': '982e0200-3f7c-11ed-bf0b-f43e93cf6d83'}),
 Document(page_content=', nonlinear, directions, ways, different, connections before seen.', metadata={'source': '..\\data\\reformatted_transcripts\\2022-09-29 11.01.46_rf.csv', 'row': 49, 'speaker_name': 'Katrin Heimann', 'date_time': 'Thu Sep 29 00:26:28 2022 ', 'uuid': '982e0200-3f7c-11ed-bf0b-f43e93cf6d83'}),
 Document(page_content=' you today, some of these parallels that I found and how I interpreted them, so that we can all learn something from it.\n\nKeywords: exhibition, interview, analyze, parallels, learn.', metadata={'source': '..\\data\\reformatted_transcripts\\2022-09-29 11.01.46_rf.csv', 'row': 49, 'speaker_name': 'Katrin Heimann', 'date_time': 'Thu Sep 29 00:26:28 2022 ', 'uuid': '982e0200-3f7c-11e

In [37]:
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

# Given list of document

# New content to replace "page_content"
new_content = "This is the new content for the page."

# Updating the page_content for each document
for document in docs_sample:
    document.page_content = new_content

# Printing the updated documents
for idx, document in enumerate(docs_sample):
    print(f"Document {idx + 1}: {document.page_content}")


Document 1: This is the new content for the page.
Document 2: This is the new content for the page.
Document 3: This is the new content for the page.
Document 4: This is the new content for the page.
Document 5: This is the new content for the page.


In [ ]:
if index_name not in pinecone_instance.list_indexes().names():
        docs = datachunk()
        pinecone_instance.create_index(name=index_name, metric="cosine", dimension=768, spec=spec)
        docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)
        print("Created new Pinecone index and loaded documents")
    else:
        docsearch = Pinecone.from_existing_index(index_name, embeddings)
        print("Using existing Pinecone index")

In [6]:
multiquery_retriever_llm = MultiQueryRetriever.from_llm(
    retriever=docsearch.as_retriever(), llm=llm
)

In [7]:
docsearch.as_retriever().invoke("What is EER?")

[Document(page_content="Helene Nymann, 2022-10-13, 19:25, that's where EER comes into the picture. No more I was thinking of like sharing perspectives or something, you know, like, what would that would that do? Or you know, that have visitors pose a question before they come in or think about, you know, could something be done that little", metadata={'source': 'data/reformatted_transcripts/2022-10-13 09.05.42_rf.txt'}),
 Document(page_content="World Wide WebBackINviteME Online'EER and the More-than-human'workshopEER | INviteME Onlinehttps://www.eer.info/activities/inviteme-online\n3 of 1028/01/2024, 18:10", metadata={'page': 2.0, 'source': 'data/EER-site-pages-pdf/EER INviteME Online.pdf'}),
 Document(page_content="Sophie Erlund, 2022-01-20, 38:52, Okay, I just wanted to say that, I think that EER, as a project, of course, it's built into it is also this thing about iterations, right? Looking at and reflecting, right, experiencing experimenting with something people haven't experience

In [9]:
default_template = """
   You are a chatbot working for the Experimenting Experiencing Reflecting (EER) Project, a research endeavor investigating the connections between art and science.
    You have access to a collection of documents, including descriptions of research activities, meeting transcripts, and other relevant materials.
    Your main task is to help the user explore and reflect on the EER project.
    All questions should pertain to the EER Project unless specified otherwise.
    When possible, please cite source documents at the end of your answer.
    """
    
template_end = """
    Context: {context}
    Question: {question}
    Answer: 
    
    """

In [11]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [10]:
rag_chain = (
    {"context": multiquery_retriever_llm, "question": RunnablePassthrough()}
    | PromptTemplate(template = default_template + template_end, input_variables=["context", "question"])
    | llm
    | StrOutputParser()
)